# **Capstone Project **
### this is The Capstone Project Jupyter Notepook

In [22]:
 !pip install --user geopy

Requirement not upgraded as not directly required: geopy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [9]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import geocoder


In [23]:
import numpy as np # library to handle data in a vectorized manner
import geocoder
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.geocoders import ArcGIS # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [24]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_page=requests.get(wikipedia_link)
page=wikipedia_page.text
soup = BeautifulSoup(page, 'lxml')
tab = soup.find('table')
tab_row = tab.find_all('tr')
data = [] # parsed data

# parse table rows
for row in tab_row:
    cols = row.find_all('td')
    cols = [element.text.strip() for element in cols]
    data.append([element for element in cols if element]) # Get rid of empty values

# parse table head
head = tab.find_all('th')
head = [element.text.strip() for element in head]
data[0]=head
data[0][2]='Neighborhood' # Change british english to american
data[0:5]

[['Postcode', 'Borough', 'Neighborhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village']]

In [26]:
import pandas as pd
df = pd.DataFrame(data[1:], columns = data[0])
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [27]:
df = df[df.Borough != 'Not assigned']

In [28]:
df.Neighborhood[df.Neighborhood == 'Not assigned'] = df.Borough[df.Neighborhood == 'Not assigned']

In [29]:
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [30]:
df['Latitude'] = 0.0
df['Longitude'] = 0.0

In [34]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [35]:
import geocoder
geolocator = ArcGIS() # Nominatim retrive NULLs based on Toronto Postal Code.
for idx in range(0,len(df)):
    address = ('{}, Toronto, Ontario'.format(df.iloc[idx]['Postcode']))
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    df['Latitude'][idx],df['Longitude'][idx]=latitude, longitude
df.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440


In [36]:
toronto_data=df